# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
758798717,2019-10-03T00:00:00Z,735109609,brain_observatory_1.1,102.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,593,1486,4,"[MGv, MGd, TH, DG, CA1, VISal, nan, SGN, LP, S..."
762602078,2019-10-03T00:00:00Z,744915204,brain_observatory_1.1,110.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,531,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, VISrl, LGv..."


In [7]:
session_id = filtered_sessions.index.values[2]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.58G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 684


,L_ratio,presence_ratio,d_prime,waveform_halfwidth,cluster_id,firing_rate,peak_channel_id,silhouette_score,waveform_repolarization_slope,waveform_amplitude,snr,waveform_velocity_below,max_drift,nn_miss_rate,nn_hit_rate,waveform_velocity_above,waveform_spread,amplitude_cutoff,waveform_duration,waveform_PT_ratio,cumulative_drift,isolation_distance,isi_violations,waveform_recovery_slope,local_index_unit,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951814884,0.024771,0.99,3.555518,0.096147,6,9.492176,850126382,0.033776,0.673650,187.434780,3.342535,0.000000,45.64,0.016202,0.727333,0.000000,40.0,0.042404,0.151089,0.522713,463.86,46.750473,0.181638,-0.249885,5,NaN,2600.0,2.370440,1.765128,3.983333,1.866667,5.186364,0.244796,0.053998,0.033134,0.023705,61.923,0.612162,0.358,0.04,15.0,-0.154286,-0.005676,-0.228819,-0.212121,-0.310345,180.0,150.0,0.619716,9.763807e-01,0.507405,0.469507,0.447263,-5.000,4929,0.75,17.571944,11.057561,5.845194,8.805963,6.974832,NaN,0.0385,False,False,False,False,False,0.042644,False,0.011829,0.000012,0.062616,0.058606,0.039197,60,43,760640083,4,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951814876,0.001785,0.99,4.445414,0.206030,5,39.100557,850126382,0.108908,0.518633,129.686505,2.589717,0.000000,40.68,0.003756,1.000000,0.000000,50.0,0.097286,0.315913,0.652514,325.21,85.178750,0.004799,-0.143762,4,NaN,900.0,3.417573,0.704762,0.672690,0.803980,1.003055,0.137709,0.017675,0.015248,0.027334,72.222,0.678473,0.795,0.04,2.0,0.326587,0.231595,0.062157,0.068548,0.068853,315.0,150.0,0.000030,6.157503e-07,0.353566,0.674185,0.472870,45.556,5021,0.00,43.390301,44.709848,36.820288,44.885084,35.195889,NaN,0.1535,False,False,False,False,False,0.214051,False,0.001812,0.000003,0.002366,0.004308,0.002943,60,43,760640083,4,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815032,0.035654,0.99,3.848256,0.096147,17,28.383277,850126398,0.096715,0.766347,207.380940,3.811566,-0.686767,40.01,0.014673,0.986000,-0.206030,80.0,0.015482,0.164824,0.484297,396.28,89.608836,0.007099,-0.255492,15,NaN,400.0,2.301810,1.408866,2.711028,1.714790,2.055258,0.173597,0.013665,0.007886,0.051909,85.000,0.768989,0.395,0.04,15.0,-0.026107,-0.220335,-0.345271,0.043011,-0.157445,135.0,120.0,0.827195,1.179300e-02,0.006750,0.867140,0.229617,5.000,4990,0.25,29.464485,28.829592,28.252666,28.025354,30.002900,NaN,0.0495,False,False,False,False,False,-0.005102,False,0.004121,0.006973,0.006743,0.018808,0.007783,140,43,760640083,12,215.0,APN,8129.0,3419.0,6762.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815275,0.016783,0.99,3.065938,0.096147,30,5.709358,850126416,0.144249,0.628944,158.158650,2.918134,0.000000,33.32,0.003683,0.883598,0.686767,60.0,0.063807,0.178559,0.600600,374.82,48.114336,0.032317,-0.206676,27,NaN,200.0,3.264583,1.145098,2.160000,1.039456,2.604037,0.380119,0.018499,0.036959,0.072179,25.000,0.013071,0.786,0.04,15.0,-0.39781

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 35


,L_ratio,presence_ratio,d_prime,waveform_halfwidth,cluster_id,firing_rate,peak_channel_id,silhouette_score,waveform_repolarization_slope,waveform_amplitude,snr,waveform_velocity_below,max_drift,nn_miss_rate,nn_hit_rate,waveform_velocity_above,waveform_spread,amplitude_cutoff,waveform_duration,waveform_PT_ratio,cumulative_drift,isolation_distance,isi_violations,waveform_recovery_slope,local_index_unit,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951810369,0.053450,0.99,3.578593,0.206030,131,5.595007,850125488,-0.008220,0.216118,83.823675,2.724453,-0.068677,73.14,0.005199,0.980000,0.206030,70.0,0.002509,0.714238,0.396781,245.89,53.841400,0.082927,-0.010352,115,NaN,1200.0,2.099090,0.678209,0.778929,1.063768,1.012987,0.176746,0.015651,0.620335,0.324444,63.333,2.118530,0.000,0.02,1.0,0.138933,-0.169056,0.292929,-0.578947,0.404075,45.0,30.0,0.232289,1.055137e-01,0.005999,0.052551,4.178670e-03,35.000,5016,0.25,7.668730,7.114141,5.168113,5.538404,3.575675,NaN,0.1115,False,False,False,False,False,0.595831,False,0.489465,0.407199,0.185159,0.139541,0.327214,2320,27,760640090,231,385.0,VISp,8860.0,1387.0,7831.0,probeC,See electrode locations,29999.905275,1249.996053,True
951810338,0.013016,0.99,4.067453,0.233501,128,6.034558,850125486,0.155323,0.185828,65.479440,2.518815,-0.686767,70.14,0.006915,0.966667,-0.206030,60.0,0.008509,0.741709,0.415458,231.28,71.210388,0.030994,-0.013399,112,NaN,100.0,2.806279,2.325614,0.482388,11.200000,0.902637,0.142526,0.015393,0.937730,0.168930,60.000,0.429992,0.437,0.32,2.0,0.199834,-0.600000,0.095001,-1.000000,0.337469,270.0,90.0,0.024679,2.034199e-01,0.154208,0.363217,7.564979e-07,0.000,4985,0.50,6.283561,0.639473,10.813810,0.163378,10.084896,NaN,0.1315,False,False,False,False,False,0.402034,True,0.725476,0.507772,0.235326,0.799812,0.380606,2320,59,760640090,230,385.0,VISp,8858.0,1397.0,7830.0,probeC,See electrode locations,29999.905275,1249.996053,True
951810425,0.018436,0.99,4.238960,0.164824,136,18.240085,850125500,-0.001033,0.692030,150.832890,2.776248,0.000000,79.06,0.002171,0.998000,0.755444,60.0,0.000043,0.247236,0.646760,113.73,87.067234,0.000339,-0.163471,120,NaN,500.0,7.432887,1.671890,1.640000,2.400000,2.761596,0.221726,0.013065,0.200051,0.083201,50.000,0.012208,0.000,0.08,1.0,0.457933,0.398525,0.357143,0.691057,0.473231,315.0,150.0,0.006358,6.013386e-07,0.004358,0.002966,3.434587e-07,10.000,5003,0.75,12.446926,25.552291,19.128870,6.382706,30.673006,NaN,0.1145,False,False,False,False,False,0.452847,False,0.149722,0.053507,0.042974,0.135472,0.040210,2380,11,760640090,237,385.0,VISp,8871.0,1324.0,7837.0,probeC,See electrode locations,29999.905275,1249.996053,True
951812218,0.037290,0.99,2.817388,0.164824,296,3.790030,850125498,0.092064,0.219161,89.132550,2.886240,0.274707,58.86,0.006228,0.878000,1.295047,90.0,0.013749,0.618090,0.657693,208.19,45.267961,0.026192,-0.060992,277,NaN,500.0,11.884337,1.577681,0.864324,2.631293,0.962622,0.293952,0.131574,0.653511,0.323128,66.000,0.36902

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951810369     3.575675
951810338    10.084896
951810425    30.673006
951812218     3.421804
951810459     7.461758
951810548     3.563685
951810495     3.161355
951810564    25.816402
951810714    32.305640
951810747    44.304935
951810810    15.327843
951810800     8.529531
951810780     9.844431
951810862    15.744162
951810906     8.471579
951810821    11.060747
951811003     2.110901
951810960    12.345672
951811011     9.470744
951811082    11.103378
951811061     4.502233
951811269     1.713899
951811217     6.466590
951811321     5.922379
951811306     4.509561
951811295     8.825283
951811426     5.207643
951812296     0.215819
951811504     8.243770
951811494     7.662923
951811535    11.477065
951811578     1.724557
951811558     1.559362
951812326     0.179850
951811789     8.478240
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

9.859066256611095


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951810369     7.668730
951810338     6.283561
951810425    12.446926
951812218     2.750746
951810459     3.092272
951810548     4.356715
951810495     0.271633
951810564    19.738653
951810714    16.893127
951810747    22.471925
951810810    13.816210
951810800     7.898003
951810780     8.263357
951810862    13.210463
951810906     6.396874
951810821    10.629157
951811003     2.267843
951810960    11.435584
951811011     8.215172
951811082    14.592455
951811061     2.206421
951811269     0.527381
951811217     2.490232
951811321     5.221916
951811306     5.810189
951811295     7.265252
951811426     2.313380
951812296     0.060363
951811504     2.864059
951811494     4.755957
951811535     5.467074
951811578     0.212858
951811558     0.793189
951812326     0.106959
951811789     3.903994
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

6.762817997788138


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])